# Lab For Experimentation

In [1]:
import warnings
from typing import Any, Literal

import numpy as np
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)


# Demo (Prevents ruff from removing the unused module import)
name: Any
category: Literal["A", "B", "C"]

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/Projects/Bike-Rental-Prediction


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

In [5]:
rng = np.random.default_rng(123)
x = rng.standard_normal(size=(1_000, 10))

X_train, X_test = train_test_split(x, test_size=0.2, random_state=123)
y_train = rng.standard_normal(size=(X_train.shape[0],))
y_test = rng.standard_normal(size=(X_test.shape[0],))

params: dict[str, Any] = {
    "n_estimators": 100,
    "max_depth": 10,
}

rf_reg = RandomForestRegressor(**params)

rf_reg.fit(X_train, y_train)
# rf_reg.score(X_test, y_test)

,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [6]:
rf_reg.feature_importances_

array([0.127 , 0.1077, 0.0703, 0.1024, 0.0978, 0.1168, 0.0876, 0.1066,
       0.0914, 0.0925])

In [7]:
import xgboost as xgb


# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [8]:
# Define hyperparameters
params = {
    "objective": "reg:squarederror",  # for regression
    "eval_metric": ["rmse", "mae", "mape"],
    "learning_rate": 0.1,
    "max_depth": 6,
    "tree_method": "hist",  # Use 'hist' for CPU, 'gpu_hist' for GPU
}
n: int = 100

# Train the model
model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
)

In [9]:
preds = model.predict(dtest_reg)
rmse = root_mean_squared_error(y_test, preds)

print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 1.125


In [10]:
# Cross-validation
cv_results = xgb.cv(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=20,
    nfold=5,
    metrics={"rmse"},
    seed=123,
    as_pandas=True,
    callbacks=[
        xgb.callback.EvaluationMonitor(show_stdv=True),
        xgb.callback.EarlyStopping(rounds=3),
    ],
)
console.print(cv_results)

[0]	train-rmse:0.97861+0.00793	test-rmse:1.00251+0.03045
[1]	train-rmse:0.95530+0.00922	test-rmse:0.99990+0.03146
[2]	train-rmse:0.93566+0.01087	test-rmse:0.99963+0.03280
[3]	train-rmse:0.91714+0.01274	test-rmse:1.00122+0.03374
[4]	train-rmse:0.90072+0.01365	test-rmse:1.00416+0.03115
[5]	train-rmse:0.88468+0.01652	test-rmse:1.00457+0.03222


train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
0         0.978613        0.007929        1.002512       0.030454
1         0.955297        0.009222        0.999901       0.031463
2         0.935665        0.010871        0.999630       0.032802

In [11]:
# Find the optimal number of boosting rounds
best_num_rounds: int = len(cv_results)
console.print(f"Optimal boosting rounds: {best_num_rounds}", style="success")
console.print(cv_results.tail())

# Step 2: Train final model with optimal rounds
final_model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=best_num_rounds,
)

# Step 3: Evaluate on test set
test_preds = final_model.predict(dtest_reg)
test_rmse = root_mean_squared_error(y_test, test_preds)
console.print(f"Test RMSE: {test_rmse:.4f}", style="info")


# Step 4: Save the model
final_model.save_model("xgboost_model.json")

Optimal boosting rounds: 3

train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
0         0.978613        0.007929        1.002512       0.030454
1         0.955297        0.009222        0.999901       0.031463
2         0.935665        0.010871        0.999630       0.032802

Test RMSE: 1.0805

In [12]:
cv_results

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,0.978613,0.007929,1.002512,0.030454
1,0.955297,0.009222,0.999901,0.031463
2,0.935665,0.010871,0.999630,0.032802


In [13]:
type(final_model).__module__

'xgboost.core'

In [14]:
import httpx

try:
    async with httpx.AsyncClient() as client:
        r = await client.get("http://0.0.0.0:5001/#/experiments/list", timeout=10.0)

    if r.status_code == 200:
        console.print("MLflow Tracking Server is reachable!", style="success")

except Exception as e:
    console.print(f"Error: {e}", style="error")

MLflow Tracking Server is reachable!

In [15]:
from src.config.config import app_config
from src.ml.trainer import ModelTrainer

In [16]:
fp: str = "../../../../Documents/data_dump/bike_data/database.parquet"
data: pl.DataFrame = pl.read_parquet(fp)
console.print(f"Shape: {data.shape}", style="info")
display(data.head(2))

trainer = ModelTrainer(data, config=app_config.feature_config)

Shape: (13903, 16)

datetime,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
str,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64
"""2011-01-01 00:00:00""",1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
"""2011-01-01 01:00:00""",1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0.0,8,32,40


2025/10/06 15:44:21 INFO mlflow.tracking.fluent: Experiment with name 'bike rental' does not exist. Creating a new experiment.


2025-10-06 15:44:21 - mlflow_tracker - [INFO] - Initialized MLFlowTracker with experiment: bike rental
2025-10-06 15:44:21 - trainer - [INFO] - Data preparation complete.


In [ ]:
from src.config import app_config
from src.ml.feature_engineering import FeatureEngineer
from src.utilities.data_validator import data_validator

In [42]:
res = data_validator(data)
console.print(res)

DataValidatorSchema(
    data_schema=DataSchema(
        numeric={
            'season': 'Int64',
            'yr': 'Int64',
            'mnth': 'Int64',
            'hr': 'Int64',
            'holiday': 'Int64',
            'weekday': 'Int64',
            'workingday': 'Int64',
            'weathersit': 'Int64',
            'temp': 'Float64',
            'atemp': 'Float64',
            'hum': 'Float64',
            'windspeed': 'Float64',
            'casual': 'Int64',
            'registered': 'Int64',
            'cnt': 'Int64'
        },
        categorical={}
    ),
    data_shape=DataShape(
        total_rows=13903,
        total_columns=16,
        number_of_numeric_columns=15,
        number_of_categorical_columns=1
    ),
    summary_statistics=SummaryStatistics(
        numeric=[
            NumericColumnStats(
                column='season',
                mean=2.26,
                median=2.0,
                mode=[2.0],
                std=1.04,
                variance=1.07,
                range=3.0,
                min=1.0,
                max=4.0,
                count=13903,
                missing_values=0,
                missing_pct=0.0,
                unique_values=4
            ),
            NumericColumnStats(
                column='yr',
                mean=0.38,
                median=0.0,
                mode=[0.0],
                std=0.48,
                variance=0.24,
                range=1.0,
                min=0.0,
                max=1.0,
                count=13903,
                missing_values=0,
                missing_pct=0.0,
                unique_values=2
            ),
            NumericColumnStats(
                column='mnth',
                mean=5.65,
                median=5.0,
                mode=[7.0, 5.0],
                std=3.22,
                variance=10.36,
                range=11.0,
                min=1.0,
                max=12.0,
                count=13903,
                missing_values=0,
                missing_pct=0.0,
                unique_values=12
            ),
            NumericColumnStats(
                column='hr',
                mean=11.55,
                median=12.0,
                mode=[16.0, 17.0],
                std=6.91,
                variance=47.78,
                range=23.0,
                min=0.0,
                max=23.0,
                count=13903,
                missing_values=0,
                missing_pct=0.0,
                unique_values=24
            ),
            NumericColumnStats(
                column='holiday',
                mean=0.03,
                median=0.0,
                mode=[0.0],
                std=0.16,
                variance=0.03,
                range=1.0,
                min=0.0,
                max=1.0,
                count=13903,
                missing_values=0,
                missing_pct=0.0,
                unique_values=2
            ),
            NumericColumnStats(
                column='weekday',
                mean=3.0,
                median=3.0,
                mode=[6.0],
                std=2.01,
                variance=4.02,
                range=6.0,
                min=0.0,
                max=6.0,
                count=13903,
                missing_values=0,
                missing_pct=0.0,
                unique_values=7
            ),
            NumericColumnStats(
                column='workingday',
                mean=0.68,
                median=1.0,
                mode=[1.0],
                std=0.46,
                variance=0.22,
                range=1.0,
                min=0.0,
                max=1.0,
                count=13903,
                missing_values=0,
                missing_pct=0.0,
                unique_values=2
            ),
            NumericColumnStats(
                column='weathersit',
                mean=1.42,
                median=1.0,
                mode=[1.0],
                std=0.64,
                variance=0.41

In [45]:
feat_eng = FeatureEngineer()
features = feat_eng.create_all_features(data, app_config.feature_config)
display(features.head(2))

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,is_weekend,sin_hour,cos_hour,sin_weekday,cos_weekday,cnt_lag_0hr,cnt_lag_1hr,cnt_lag_24hr,hr_lag_1hr,hr_lag_24hr,temp_lag_1hr,temp_lag_3hr,hum_lag_1hr,hum_lag_3hr,temp_rolling_mean_3hr,temp_rolling_median_3hr,temp_rolling_mean_6hr,temp_rolling_median_6hr,hum_rolling_mean_3hr,hum_rolling_median_3hr,hum_rolling_mean_6hr,hum_rolling_median_6hr,temp_plus_hum,hum_plus_hr,cnt_diff_1hr,cnt_diff_2hr,hr_diff_1hr,hr_diff_24hr,temp_diff_1hr,temp_diff_2hr,temp_diff_24hr,hum_diff_1hr,hum_diff_2hr,is_high_temp,is_high_hum,is_peak_hour,is_working_hour,is_business_hour,target
i64,i64,i64,i64,i64,i64,i64,f64,f64,i8,f64,f64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i8,i8,i8,i8,i8,i64
1,1,0,0,6,0,1,0.24,0.81,1,0.0,1.0,-0.781831,0.62349,16,16,16,0,0,0.24,0.24,0.81,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.05,0.81,24,16,1,0,-0.02,-0.02,0.22,-0.01,-0.01,0,0,0,0,0,40
1,1,1,0,6,0,1,0.22,0.8,1,0.258819,0.965926,-0.781831,0.62349,40,16,16,0,0,0.24,0.24,0.81,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.02,1.8,24,16,1,0,-0.02,-0.02,0.22,-0.01,-0.01,0,0,0,0,0,32


In [ ]:
trainer.train_all_models()

In [ ]:
import mlflow

mlflow.active_run().info.run_id